In [100]:
import pandas as pd

In [101]:
df_de = pd.read_csv('../grams/sg_de.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams
df_nl = pd.read_csv('../grams/sg_nl.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams
df_fr = pd.read_csv('../grams/sg_fr.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams
df_es = pd.read_csv('../grams/sg_es.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams

In [102]:
# list of ~94k US babynames to remove (hello, john), (james, bond)
names = pd.read_csv("../names.csv", names=['name'])['name'].tolist()

In [103]:
names = [i for i in names if type(i) == str] #remove non strings

# Filter names and from list

In [104]:
german_filter = ['ich', 'du', 'sie', 'mir', 'wir', 'ihr', 'uns', 'euch', 'die', 'das', 'der', 'den', 'er', 'und', '!', '/', 'ein', 'eine', 'einer', 'einem', 'mein', 'dein', 'nicht', 'wie', 'wo', 'wann', 'was']

In [105]:
dutch_filter = ['ik', 'jij', 'hij', 'zij', 'wij', 'we', 'jullie', 'hun', 'mij', 'me', 'mijn', 'haar', 'hem', '!', '/', 'een', 'de', 'het', 'wie', 'wat', 'waar', 'hoe', 'niet']

In [106]:
french_filter = ['un', 'une', 'le', 'la', '!', '/', 'je', 'tu', 'nous', 'vous']

In [107]:
spanish_filter = ['!', '/', 'el']

In [108]:
l_de = names + german_filter
l_nl = names + dutch_filter
l_fr = names + french_filter
l_es = names + spanish_filter


df_fr.word1 = df_fr.word1.apply(str)
df_fr.word2 = df_fr.word2.apply(str)
df_es.word1 = df_es.word1.apply(str)
df_es.word2 = df_es.word2.apply(str)

In [109]:
df_de = \
    df_de[ \
    ~df_de.word1.isin(l_de) & \
    ~df_de.word2.isin(l_de) & \
    ~df_de.word1.str.isnumeric() & \
    ~df_de.word2.str.isnumeric() &\
    ~(df_de.word1.str == df_de.word2.str)
    ]

df_nl = \
    df_nl[ \
    ~df_nl.word1.isin(l_nl) & \
    ~df_nl.word2.isin(l_nl) & \
    ~df_nl.word1.str.isnumeric() & \
    ~df_nl.word2.str.isnumeric() & \
    ~(df_de.word1.str == df_de.word2.str)
    ]

df_fr = \
    df_fr[ \
    ~df_fr.word1.isin(l_fr) & \
    ~df_fr.word2.isin(l_fr) & \
    ~df_fr.word1.str.isnumeric() & \
    ~df_fr.word2.str.isnumeric() & \
    ~(df_fr.word1.str == df_fr.word2.str)
    ]
    
df_es = \
    df_es[ \
    ~df_es.word1.isin(l_es) & \
    ~df_es.word2.isin(l_es) & \
    ~df_es.word1.str.isnumeric() & \
    ~df_es.word2.str.isnumeric() & \
    ~(df_es.word1.str == df_es.word2.str)
    ]

# Remove words overlapping with other languages

In [115]:
df_de_ex = df_de \
    .merge(df_nl ,left_on=['word1', 'word2'], right_on=['word1', 'word2'], how='left') \
    .merge(df_fr ,left_on=['word1', 'word2'], right_on=['word1', 'word2'], how='left') \
    .merge(df_es ,left_on=['word1', 'word2'], right_on=['word1', 'word2'], how='left')

In [116]:
df_de_ex = df_de_ex.fillna(0)

In [117]:
df_de_ex.columns = ['frequency', 'word1', 'word2', 'nl', 'fr', 'es']

In [118]:
df = df_de_ex[(df_de_ex.nl == 0.0) & (df_de_ex.fr == 0.0) & (df_de_ex.es == 0.0)].drop(['nl', 'fr', 'es'], axis=1)

# Add word frequency

In [120]:
wc_de = pd.read_csv("../word_counts/wc_de.csv", lineterminator='\n', names=['word', 'word_frequency'])

In [121]:
df = df.merge(wc_de, left_on='word1', right_on='word', how='left') \
.merge(wc_de, left_on='word2', right_on='word', how='left') \
.drop(['word_x', 'word_y'], axis=1) \
.rename(index=str, columns={"word_frequency_x": "word_1_frequency", "word_frequency_y": "word_2_frequency"})

In [122]:
df['normalised_frequency'] = df['frequency'] / (df['word_1_frequency']+df['word_2_frequency']) # divide skipgram freq. by wordcount freq.

In [123]:
df = df.sort_values(by='frequency', ascending=False)

In [125]:
df.count()

frequency               274238
word1                   274238
word2                   274238
word_1_frequency        274238
word_2_frequency        274238
normalised_frequency    274238
dtype: int64